In [1]:

using POMDPs, QuickPOMDPs, MCTS, DiscreteValueIteration, POMDPSimulators, POMDPModels, POMDPPolicies, POMDPModelTools
using Distributions, Combinatorics, StaticArrays, Statistics
using FileIO, JLD2, TickTock

## State Functions

In [2]:
function state_cnt(n, S) #n = number of units; S = number of states
    if S==1
        return 1
    end
    return binomial(n+S-1, S-1)
end

function state_index(S,s) #s = state vector
    #S = length(s)
    if S == 1
        return 1
    end
    if s[S]==0
        return state_index(S-1, s[1:(S-1)])
    end
    
    n_prev = sum(s)
    prev = state_cnt(n_prev, S-1) # count of all states with s[S]=0
    inc = prev
    for i in 1:(s[S]-1)
        inc = inc/(n_prev+S-2)*n_prev #count of all states with s[S]=i
        prev = prev + inc
        n_prev = n_prev - 1
    end
    return prev + state_index(S-1, s[1:(S-1)])
end

function state_vec(n, S, ind) # n = number of units; S = number of states; ind = state index
    if ind < 0.5 || ind > state_cnt(n, S) + 0.5
        println("index is out of range!")
        return -1
    end
    if S==1 
        return [n]
    end
    if ind < state_cnt(n, S-1) + 0.5
        return push!(state_vec(n, S-1, ind), 0)
    end
    prev = state_cnt(n, S-1)
    inc = prev
    last_state = 0
    n_prev = n
    while ind > prev + 0.5
        inc = inc/(n_prev+S-2)*n_prev #count of all sta_tes with s[S]=last_state
        prev += inc
        n_prev -= 1
        last_state += 1
    end
    return push!(state_vec(n-last_state, S-1, ind - prev + inc), last_state)
end

#using hueristic rule for rollout
mutable struct nNRollout <: Policy
    n::Int64
    N::Int64
end

mutable struct nmNRollout <: Policy
    n::Int64
    m::Int64
    N::Int64
end

mutable struct mystate
    state::Vector{Int64};
end

In [3]:
function POMDPs.action(p::nNRollout, s::mystate)
        local_a = 0;
        local_s = s.state;
        nN = sample(1:Number_level, 2, replace = true);
        p.N = maximum(nN);
        p.n = minimum(nN);
        if sum(local_s[p.N:Number_level])>=1
        local_a = p.n;
        end
        return local_a; 
end

function POMDPs.action(p::nmNRollout, s::mystate)
    local_a = 0;
    local_s = s.state;
    nmN = sample(1:Number_level, 3, replace = true);
    p.N = maximum(nmN);
    p.n = minimum(nmN);
    p.m = sum(nmN)-p.N-p.n;
    if sum(local_s[p.N:Number_level])>=1 || sum(local_s[p.m:Number_level])>=2
        local_a = p.n;
    end
    return local_a; 
end

In [4]:

function findnmN()
    println("Finding best (n, m, N) policy...")
    for N in 2:(Number_level)
        for n in 1:Number_level
            for m in n:N
                trials=10000;
                simsteps = 100;
                results=zeros(trials)*0.1
                nmN=[n,m,N]
                Threads.@threads for h in 1:trials
                s=repeat(1:1,NumberUnits)
                r1=[]
                a1=[]
                
                r=repeat(1:1,NumberUnits)*1.0
                 for k in 1:simsteps
                    a =decison(nmN,s)
                            r2=0.0
                    if 1 in a
                    for i in 1:NumberUnits
                         s[i],r[i]=generative(s[i],a[i]+2,i,234);
                          r2+=r[i]
                            end 
                    else
                       for i in 1:NumberUnits
                         s[i],r[i]=generative(s[i],1,i,234);
                          r2+=r[i]
                            end              
                    end
                    s=s;
                    append!(a1,a)
                    append!(r1,r2)
                end
                rk=0
                for i in 1:simsteps
                    rk=0.95*rk+r1[simsteps-i+1]
                end
                    results[h]=rk
                end
                rewards_nmN[n, m, N] = mean(results);
                rewards_nmN_std[n, m, N] = var(results)^(0.5);
            end
        end
    end
    (max_rward,nmN) = findmax(rewards_nmN);
    println("Max rewards is ", max_rward, "  n is ", nmN[1], " m is ", nmN[2], "  N is ",nmN[3]);
    return max_rward,rewards_nmN_std[ nmN[1], nmN[2],nmN[3]],nmN;    
end
 


findnmN (generic function with 1 method)

In [5]:
function decison(nmN, s,n=0.2)
   a_final=[]
    if length(s[s .>= nmN[3]])>0 || length(s[s .>= nmN[2]])>1
        for i in 1:NumberUnits
            append!(a_final,Int(s[i]>=nmN[1]))
            end
        return a_final
    end
    a_0=repeat(0:0,NumberUnits);
     return a_0
end
Number_level=10
T=zeros(Number_level,Number_level,3,100);

for i in 1:100
    fullname = "TM_MATRIX/tm"*string(i);
    Transition_matrix1 = load(fullname*".jld2","transition_matrix");
    
    T[:,:,1,i].=Transition_matrix1;
    T[:,:,2,i].=Transition_matrix1;
    for j in 1:10
        T[j,:,3,i].=Transition_matrix1[1,:];
        end
    T[10,:,1,i].=Transition_matrix1[1,:]
T[10,:,2,i].=Transition_matrix1[1,:]
end

function generative(s, a,k, rng=23)       #s is a vector of number units at each level and a is the number of units we will repair
        if s==Number_level
        s=1
        crd = Categorical(T[s,:,1,k]);
            s = rand(crd);
            r=failure_penalty +  setup_cost/NumberUnits + normal_operation  ;
        return (sp=s, r=r)
       end
        r=0
     if a==1
            crd = Categorical(T[s,:,1,k]);
            s = rand(crd);
            r = normal_operation ;
         return (sp=s, r=r)
        end
    if a==2
            crd = Categorical(T[s,:,1,k]);
            s = rand(crd);
            r = normal_operation +  setup_cost/NumberUnits ;
             return (sp=s, r=r)
        end
    if a==3
        s1=deepcopy(s)
        s=1
        r = maintenance_penalty +  setup_cost/NumberUnits
         crd = Categorical(T[s,:,1,k]);
            s = rand(crd);
            r+= normal_operation  ;
       return (sp=s, r=r)
    end
    end

generative (generic function with 2 methods)

In [6]:
using DataFrames
df= DataFrame(u=[],n=[],s=[],m=[],f=[],mean=[],nmN=[],std=[])

cost=[ 
    
    [0 -1200 -200 -1000]
    ]

for units in [20]
    global NumberUnits=convert(Int64,units)
global Number_level = 10;
global fullname = "tm10.jld2";
fullname = "tm10";

Transition_matrix = mean(T[:,:,1,1:3],dims=3);
Transition_matrix=(Transition_matrix./sum(Transition_matrix,dims=2))
global state_number = state_cnt(NumberUnits,Number_level);


global crd = Array{Categorical}(undef,Number_level)
for i in 1:Number_level
    global crd[i] = Categorical(Transition_matrix[i,:]);
end
    
    
    
global multiunit2 = QuickMDP(
    gen = function (s, a, rng)       #s is a vector of number units at each level and a is the number of units we will repair
        local_s = s.state;
        degradation_state = repeat(1:1,NumberUnits);
        k = 1;
        for i in 1:Number_level
            for j in 1:local_s[i]
                degradation_state[k]=i;
                k = k+1;
            end
        end
        r = 0.0;
        prevent_repair = false;
        if a!=0
        number_reset = sum(local_s[a:Number_level]);
        else
        number_reset = local_s[Number_level];
        end
        #using a for loop to compute next state for each unit

        for i in 1:(NumberUnits-number_reset)  #a is the number of units we want to preventively repair
            #in this loop, all units continues
            degradation_state[i] = rand(crd[degradation_state[i]]);
            r = r+normal_operation;
        end
        
        for i in (NumberUnits-number_reset+1):NumberUnits
            if degradation_state[i] == Number_level
                r = r + failure_penalty;
                if prevent_repair == false
                    r = r+setup_cost;
                    prevent_repair = true;
                end
            else
                r = r + maintenance_penalty;
                if prevent_repair == false
                    r = r+setup_cost;
                    prevent_repair = true;
                end
            end
            degradation_state[i] = rand(crd[1]); #reset status; add additional transition
            r = r+normal_operation; #add operation benefit
         end
        #collect degradation state to form the state
        sp = repeat(0:0,Number_level);
        for i in 1:NumberUnits
            sp[degradation_state[i]] = sp[degradation_state[i]]+1;
        end
        return (sp=mystate(sp), r=r)
    end,
    actions = 0:(Number_level-1), 
    actiontype = function()
        return Int64;
    end,
#     states = arrayofstates,
    initialstate = function()
        POMDPModelTools.ImplicitDistribution() do rng
            return (mystate(state_vec(NumberUnits, Number_level, 1)))
        end
    end, #all u #all units start fresh. Need to change according to unit number and level number. ##For simulation, we need to use ImplicitDistribution
    discount = 0.95,
    isterminal = false              # no ending
)


    
    
    
for c in cost

   print(units)  
    global normal_operation,setup_cost,maintenance_penalty,failure_penalty=c
     println([normal_operation,setup_cost,maintenance_penalty,failure_penalty])     
   global  simsteps = 100;
global repetition = 10000;
global rewards_nmN = zeros(Number_level,Number_level,Number_level);
global rewards_nmN = rewards_nmN.+(-100000000.0);
global rewards_nmN_std = zeros(Number_level,Number_level,Number_level);
global rewards_nmN_std = rewards_nmN_std.+(-100000000.0);
global discount_factor =0.95
global temp_rewards = zeros(repetition,1);
rewards,std, nmN=findnmN()
tick()
trials=10000;
simsteps = 100;
results=zeros(trials)*0.1
Threads.@threads for h in 1:trials
s=repeat(1:1,NumberUnits)
r1=[]
a1=[]   
r=repeat(1:1,NumberUnits)*1.0
 for k in 1:simsteps
    a =decison(nmN,s)
            r2=0.0
    if 1 in a
    for i in 1:NumberUnits
         s[i],r[i]=generative(s[i],a[i]+2,i,234);
          r2+=r[i]
            end 
    else
       for i in 1:NumberUnits
         s[i],r[i]=generative(s[i],1,i,234);
          r2+=r[i]
            end              
    end
    s=s;
    append!(a1,a)
    append!(r1,r2)
end
rk=0
for i in 1:simsteps
    rk=0.95*rk+r1[simsteps-i+1]
end
    results[h]=rk
end
println(units)

println(mean(results))

println(var(results)^0.5)
pushfirst!(df,[units,normal_operation,setup_cost,maintenance_penalty,failure_penalty,nmN,mean(results),var(results)^0.5])
        
tock() 
    end
    
    
end

20[0, -1200, -200, -1000]
Finding best (n, m, N) policy...
Max rewards is -45447.512685887865  n is 6 m is 10  N is 10


┌ Info:  started timer at: 2023-04-17T11:56:47.738
└ @ TickTock C:\Users\vbansal5\.julia\packages\TickTock\fGILW\src\TickTock.jl:54


20
-45439.81492338397
2871.951050625186


┌ Info:            21.680291s: 21 seconds, 680 milliseconds
└ @ TickTock C:\Users\vbansal5\.julia\packages\TickTock\fGILW\src\TickTock.jl:62


In [7]:

df

Row,u,n,s,m,f,mean,nmN,std
,Any,Any,Any,Any,Any,Any,Any,Any
1,20,0,-1200,-200,-1000,"CartesianIndex(6, 10, 10)",-45439.8,2871.95


In [8]:
using CSV
CSV.write("nmN_hetero.csv", df)

"nmN_hetero.csv"